**Librerias a usar en el proceso de ETL**

In [ ]:
#pip install fuzz
#pip install python-Levenshtein
#!pip install fuzzywuzzy

In [1]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import ast
from fuzzywuzzy import fuzz,process
import functools
pd.set_option('display.max_columns', None)

## 1- ETL de Archivos YELP

### 1.1- Archivo checkin.json

**Abrimos el archivo con la siguiente función:**

In [ ]:
#Funcion para abrir chekin de yelp

def abrir_Archivo_json(archivo):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    with open(archivo) as file:
        for line in file:
            try:
                obj = json.loads(line)
                merged_data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el archivo {archivo}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
archivo = '../checkin.json'
df_checkin_yelp = abrir_Archivo_json(archivo)

**Revisamos si tiene nulos**

In [ ]:
df_checkin_yelp.isnull().sum()

**Revisamos si tiene duplicados**

In [ ]:
df_checkin_yelp.duplicated().sum()

**Desanidamos la fecha (en Fecha y hora)**

In [ ]:
desanidados = []

for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        time, date = date.split(' ')
        desanidados.append([business_id, time, date])

df_desanidado = pd.DataFrame(desanidados, columns=['business_id', 'hour', 'fecha'])

**Finalmente se exporta**

In [ ]:
df_desanidado.to_csv('checkin.csv', index=False)

### 1.2- Archivo user.parquet

**Leemos el archivo**

In [ ]:
archivo = "../user.parquet"
df_User_yelp = pd.read_parquet(archivo)

**Eliminamos duplicados y reindexamos**

In [ ]:
df = df_User_yelp.drop_duplicates(subset=["user_id", "name"], keep="first")
df_User_yelp = df_User_yelp.reindex(df.index)
df_User_yelp['id_user'] = df.index

**Creamos y exportamos un nuevo dataframe para conservar id_user original con su user_id , el cual será usado para unir tablas**

In [ ]:
dfusuario = df_User_yelp.loc[:, ["id_user",'user_id']]

In [ ]:
#Renombramos la columna user_id

dfusuario.rename(columns={"user_id": "yelp_id"}, inplace=True)

In [ ]:
#Exportamos el nuevo dataframe llamado dfusuario
dfusuario.to_csv("UsuarioYelp")

**Eliminamos la columna user_id del dataframe df_user_yelp**

In [ ]:
df_User_yelp.drop(columns=['user_id'], inplace=True)

**Aplicamos transformaciones a los tipos de datos en las diferentes variables**

In [ ]:
#Cambiamos a string la variable name

df_User_yelp['name'] = df['name'].astype(str)

#cambiamos a tipo datetime la variable yelping_since
df_User_yelp['yelping_since'] = pd.to_datetime(df['yelping_since'])

#Creamos una columna año a partir de elite que devuelva una lista de años separado por comas
df_User_yelp['years'] = df['elite'].str.split(',')

#creamos un nuevo DataFrame con filas individuales para cada año y usuario
years_df = df_User_yelp[['id_user', 'years']].explode('years')

#reemplazamos valores vacios por np.nan en years
years_df['years'] = years_df['years'].replace("", np.nan)

#reemplazamos 20 po 2020 en years
years_df['years'] = years_df['years'].replace("20", "2020")

**Exportamos segundo dataframe de user_yelp como usuarioElite**

In [ ]:
years_df.to_csv("Datasets/UsuarioElite")

**Eliminamos las columnas elite y years del dataframe original user.parquet**

In [ ]:
df_User_yelp.drop(columns=['elite'], inplace=True)
df_User_yelp.drop(columns=['years'], inplace=True)

## 1.3 Archivo business.pkl

**Leemos el archivo**

In [ ]:
df_business= pd.read_pickle('../business.pkl')

**Agregamos nuevas columnas al dataframe con valores iniciales en NONE**

In [ ]:
df_business['NAME']=None
df_business['REVIEW_COUNT']=None
df_business['POSTAL_CODE']=None
df_business['CITY']=None
df_business['STATE']=None
df_business['BUSINESS_ID']=None
df_business['ADDRESS']=None
df_business['LATITUDE']=None
df_business['LONGITUDE']=None
df_business['STARS']=None
df_business['IS_OPEN']=None
df_business['ATTRIBUTES']=None
df_business['CATEGORIES']=None
df_business['HOURS']=None

**Obtenemos la actualización de la columna "NAME" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.**

In [ ]:
for index,i in enumerate(df_business.name.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['NAME'][index]=''.join(arr)

**Obtenemos la actualización de la columna "CITY" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.**

In [ ]:
for index,i in enumerate(df_business.city.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CITY'][index]=''.join(arr)

**Obtenemos la actualización de la columna "STATE" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.**

In [ ]:
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['STATE'][index]=''.join(arr)

**En este script, se actualiza la columna 'REVIEW_COUNT' del DataFrame df_business con el primer número entero encontrado en cada valor, recorriendo los valores de la columna y almacenando los números enteros en una lista antes de asignarlos a la columna.**

In [ ]:
for index,i in enumerate(df_business.review_count.values):
    arr=[]
    for e in i:
        if isinstance(e,int):
         arr.append(e)
    df_business['REVIEW_COUNT'][index]=arr[0]

**Se fusionan los caracteres de texto presentes en cada valor de la columna 'business_id' del DataFrame df_b, actualizando la columna "BUSINESS_ID" con los valores resultantes**

In [ ]:
for index,i in enumerate(df_business.business_id.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['BUSINESS_ID'][index]=''.join(arr)


**Se fusionan los caracteres de texto presentes en cada valor de la columna 'address' del DataFrame 'df_business', actualizando la columna 'ADDRESS' con los valores resultantes**

In [ ]:
for index,i in enumerate(df_business.address.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['ADDRESS'][index]=''.join(arr)

**Se fusionan los caracteres de texto presentes en cada valor de la columna 'postal_code' del DataFrame 'df_business', actualizando la columna 'POSTAL_CODE' con los valores resultantes**

In [ ]:
for index,i in enumerate(df_business.postal_code.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['POSTAL_CODE'][index]=''.join(arr)

**Se filtran los elementos numéricos mayores que 1 en cada valor de la columna 'latitude' del DataFrame 'df_business'. A continuación, se asigna el primer elemento filtrado a la columna 'LATITUDE' en el DataFrame 'df_business' en la fila correspondiente al índice actual.**

In [ ]:
for index,i in enumerate(df_business.latitude.values):
    arr=[]
    for e in i:
       if e>1:
         arr.append(e)
    df_business['LATITUDE'][index]=arr[0]

**Se filtran los elementos numéricos menores que -1 en cada valor de la columna 'longitude' del DataFrame 'df_business'. Luego, se asigna el primer elemento filtrado a la columna 'LONGITUDE' en el DataFrame 'df_business' en la fila correspondiente al índice actual.**

In [ ]:
for index,i in enumerate(df_business.longitude.values):
    arr=[]
    for e in i:
        if e<-1:
            arr.append(e)
    df_business['LONGITUDE'][index]=arr[0]

**Se filtran los elementos numéricos mayores que 0.1 en cada valor de la columna 'stars' del DataFrame 'df_business'. Después, se asigna el primer elemento filtrado a la columna 'STARS' en el DataFrame 'df_business' en la fila correspondiente al índice actual.**

In [ ]:
for index,i in enumerate(df_business.stars.values):
    arr=[]
    for e in i:
       if e>0.1:
         arr.append(e)
    df_business['STARS'][index]=arr[0]

**Se filtran los elementos numéricos mayores o iguales a 0 en cada valor de la columna 'is_open' del DataFrame 'df_business'. A continuación, se asigna el primer elemento filtrado a la columna 'IS_OPEN' en el DataFrame 'df_business' en la fila correspondiente al índice actual**

In [ ]:
for index,i in enumerate(df_business.is_open.values):
    arr=[]
    for e in i:
       if e >=0:
         arr.append(e)
    df_business['IS_OPEN'][index]=arr[0]

**Este código busca y guarda el primer diccionario encontrado en la columna 'attributes' del dataframe 'df_business' en la columna 'ATTRIBUTES'.**

In [ ]:
for index,i in enumerate(df_business.attributes.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:
     df_business['ATTRIBUTES'][index]=arr[0]

**Este código filtra los elementos de tipo cadena en la columna 'categories' del dataframe 'df_business'. Luego, fusiona todas las cadenas filtradas en una sola cadena y la asigna a la columna 'CATEGORIES' en el dataframe 'df_business'.**

In [ ]:
for index,i in enumerate(df_business.categories.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CATEGORIES'][index]=''.join(arr)

**En este código, se recorren los valores de la columna 'hours' en el dataframe 'df_business' y se filtran los elementos que son diccionarios. Luego, se asigna el primer diccionario encontrado a la columna 'HOURS' en el dataframe 'df_business' en la fila correspondiente. En resumen, el código extrae y guarda el primer diccionario encontrado en la columna 'hours' del dataframe 'df_business' en la columna 'HOURS'.**

In [ ]:
for index,i in enumerate(df_business.hours.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:     
     df_business['HOURS'][index]=arr[0]

**Filtramos el dataframe solo a las columnas nuevas generadas con los bucles, con los nombres iniciales**

In [ ]:
df_business=df_business[['BUSINESS_ID','NAME','REVIEW_COUNT','CITY','STATE','ADDRESS','POSTAL_CODE','LATITUDE','LONGITUDE','STARS','IS_OPEN','ATTRIBUTES','CATEGORIES','HOURS']]

In [ ]:
df_business['business_id']=df_business['BUSINESS_ID']
df_business['name']=df_business['NAME']
df_business['address']=df_business['NAME']
df_business['city']=df_business['CITY']
df_business['state']=df_business['STATE']
df_business['postal_code']=df_business['POSTAL_CODE']
df_business['latitude']=df_business['LATITUDE']
df_business['longitude']=df_business['LONGITUDE']
df_business['stars']=df_business['STARS']
df_business['review_count']=df_business['REVIEW_COUNT']
df_business['is_open']=df_business['IS_OPEN']
df_business['attributes']=df_business['ATTRIBUTES']
df_business['categories']=df_business['CATEGORIES']
df_business['hours']=df_business['HOURS']
df_business=df_business[['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours']]

In [ ]:
df_business2 = df_business.copy()

**Cargamos un nuevo csv con las ciudades de estados unidos**

In [ ]:
city=pd.read_csv('../ciudades.csv')

In [ ]:
#ciudades unicas de estados unidos
ciudades_estados_unidos = city['City'].unique()

**la primera función busca la mejor coincidencia de una ciudad en una lista, mientras que la segunda función calcula un umbral de similitud basado en la longitud de una ciudad. Ambas funciones se utilizan en conjunto para determinar la mejor coincidencia de una ciudad y aplicar un criterio de aceptación basado en el umbral de similitud.**

In [ ]:
import functools

@functools.lru_cache(maxsize=None)  
def encontrar_mejor_coincidencia(ciudad):
    mejor_coincidencia = process.extractOne(ciudad, ciudades_estados_unidos)
    resultado = mejor_coincidencia[0] if mejor_coincidencia[1] >= calcular_umbral_similitud(len(ciudad)) else ciudad
    return resultado

def calcular_umbral_similitud(longitud_ciudad):
    umbral_base = 55
    umbral = umbral_base - (longitud_ciudad // 3)
    return max(umbral, umbral_base)

**Se aplica la función encontrar_mejor_coincidencia a cada valor de la columna 'city' del DataFrame df_b. La función busca la mejor coincidencia de cada ciudad en una lista de ciudades de EE.UU. y actualiza la columna 'city' con las mejores coincidencias encontradas.**

In [ ]:
df_business2['city']=df_business2['city'].apply(encontrar_mejor_coincidencia)

In [ ]:
#probamos la funcion
encontrar_mejor_coincidencia('nryork')

**Generamos un dataframe desde city con las columnas City y State solamente**

In [ ]:
citystate=city[['City','State']]

**Se fusionan dos DataFrames ('df_business' y 'citystate') por la columna 'city' en una unión izquierda.**

In [ ]:
df_business2=df_business2.merge(citystate,left_on='city',right_on='City',how='left')

In [ ]:
# Eliminar la columna duplicada 'state'
df_business2.drop('state', axis=1, inplace=True)

In [ ]:
df_business2.head(1)

**se actualiza el dataframe df_business, manteniendo solo las columnas mencionadas.**

In [ ]:
df_business2=df_business2[['business_id', 'name', 'address', 'city', 'State',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours']]

**Renombramos la columna State a state**

In [ ]:
df_business2=df_business2.rename(columns={'State':'state'})

In [ ]:
df_business2.head(1)

**Filtramos el dataframe con los 5 estados que se usarán en el proyecto**

In [ ]:
statein=['California','New York' ,'Florida' ,'Pennsylvania' ,'Texas']
df_business2=df_business2[df_business2['state'].isin(statein)].reset_index()

In [ ]:
df_business2.head(1)

**Filtramos las columnas que se van a usar en el proyecto y renombramos index**

In [ ]:
df_business2=df_business2[['index','business_id', 'name', 'address', 'city',
       'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours']]
df_business2=df_business2.rename(columns={'index':'id_business'})

**Generamos un nuevo dataframe solo con las columnas 'id_business','business_id'**

In [ ]:
#Creamos la tabla de dimension BusinessYelp
BusinessYelp = df_business2[['id_business', 'business_id']]

# Renombrar la columna "business_id" como "businessYelp_id"
BusinessYelp.rename(columns={"business_id": "businessYelp_id"}, inplace=True)

In [ ]:
BusinessYelp.head(1)

In [ ]:
#Exportamos la tabla
BusinessYelp.to_csv('BusinessYelpId.csv',index=False)

**Filtramos por ultima vez el dataframe**

In [ ]:
df_business2=df_business2[['id_business','name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours']]

In [ ]:
df_business2.head(1)

**Convertimos a númerico la columna postal_code**

In [ ]:
df_business2["postal_code"] = df_business2["postal_code"].apply(pd.to_numeric, errors='coerce')

**Creamos un dataframe auxiliar  que contiene solo las filas donde el valor de la columna 'state' coincide con "Pennsylvania", "New York", "Florida", "California" o "Texas".**

In [ ]:
aux = df_business2.query('state == "Pennsylvania" or state == "New York" or state == "Florida" or state == "California" or state == "Texas"')

**Buscamos filas específicas en el DataFrame y realizamos cambios en las columnas 'state' y 'postal_code' en función de los valores de las columnas 'city' y 'postal_code'.**

In [ ]:
df_business2.loc[(df_business2["city"] == "Philadelphia") & (df_business2["postal_code"].isnull()), "state"] = "Pennsylvania"
df_business2.loc[(df_business2["city"] == "Philadelphia") & (df_business2["postal_code"].isnull()), "postal_code"] = 19107
df_business2.loc[df_business2["city"] == "Saint Petersburg", "city"] = "St. Petersburg"
df_business2.loc[(df_business2["city"] == "St. Petersburg") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "St. Petersburg") & (df_business2["postal_code"].isnull()), "postal_code"] = 33707
df_business2.loc[(df_business2["city"] == "Tampa") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "Tampa") & (df_business2["postal_code"].isnull()), "postal_code"] = 33610
df_business2.loc[(df_business2["city"] == "Santa Barbara") & (df_business2["postal_code"].isnull()), "state"] = "California"
df_business2.loc[(df_business2["city"] == "Santa Barbara") & (df_business2["postal_code"].isnull()), "postal_code"] = 93101
df_business2.loc[(df_business2["city"] == "Pasco") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "Pasco") & (df_business2["postal_code"].isnull()), "postal_code"] = 33544

**Exportamos df_business2 para el proyecto**

In [ ]:
df_business2.to_csv('Business.csv',index=False)

**Eliminamos algunas columnas del dataframe auxiliar**

In [ ]:
aux.drop(columns=['is_open'], inplace=True)
aux.drop(columns=['postal_code'], inplace=True)

**Eliminamos duplicados de la columna auxiliar y reindexamos**

In [ ]:
aux = aux.drop_duplicates(subset=["id_business", "name"], keep="first")
aux['id_business'] = aux.index

In [ ]:
aux.columns

**Generamos un nuevo dataframe con "id_business",'categories'**

In [ ]:
#Tabla de dimension de categorias
df_cat = aux.loc[:, ['id_business','categories']]

In [ ]:
# Dividir los valores de la columna "categories" por coma y expandirlos en filas
df_cat['categories'] = df_cat['categories'].str.split(',')
df_cat = df_cat[['id_business', 'categories']].explode('categories')

In [ ]:
#Creamos un array con las categorias unicas
categorias = df_cat["categories"].unique()
#Creamos la tabla de dimension de categorias de yelp
df_categorias = pd.DataFrame(categorias, columns=["Descripcion"])
df_categorias['IdCategoria'] = df_categorias.index

**Exportamos la tabla de dimensiones de categorias**

In [ ]:
df_categorias.to_csv("Categorias.csv")

**Creamos una nuevo dataframe auxiliar para el detalle de las categorias a partir de df_Cat y df_categorias**

In [ ]:
df_aux = df_cat.merge(df_categorias, left_on="categories", right_on="Descripcion", how="inner")

**Eliminamos columnas categories y Descripcion para generar una tabla de dimension entre id_business y idcategoria**

In [ ]:
df_aux.drop(columns=["categories"],inplace=True)
df_aux.drop(columns=["Descripcion"],inplace=True)

**Exportamos tabla de dimensiones de detalle de categorias**

In [ ]:
df_aux.to_csv("DetalleCategorias.csv")

In [ ]:
aux.head(1)

**Eliminamos categories de aux**

In [ ]:
aux.drop(columns=["categories"],inplace=True)

**Generamos tabla de dimension atributos a partir de aux**

In [ ]:
df_atribute = aux.loc[:, ["id_business",'attributes']]

**Procesamos y dividimos los elementos de la columna 'attributes'**

In [ ]:
for index, i in enumerate(df_atribute['attributes']):
    if isinstance(i,str):
       df_atribute['attributes'][index]= i[1:-1].split(',')

**Expandimos la columna 'attributes' del DataFrame 'df' en filas separadas, manteniendo los valores correspondientes de la columna 'id_business'.**

In [ ]:
df_atribute = df_atribute[['id_business', 'attributes']].explode('attributes')

**Eliminamos los corchetes y comillas dobles de la columna 'attributes' del DataFrame, dejando los valores limpios y sin esos caracteres específicos**

In [ ]:
df_atribute['attributes'] = df_atribute['attributes'].str.replace('{', '').str.replace('}', '')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('"', '')

**modificamos los valores en la columna 'attributes', actualizando ciertos aspectos relacionados con estacionamiento de negocios**

In [ ]:
df_atribute['attributes'] = df_atribute['attributes'].str.replace('street: True', 'BusinessParking street: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('street: False', 'BusinessParking street: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('validated: True', 'BusinessParking validated: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('validated: False', 'BusinessParking validated: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('lot: True', 'BusinessParking lot: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('lot: False', 'BusinessParking lot: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('valet: True', 'BusinessParking valet: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('valet: False', 'BusinessParking valet: False')

**Creamos un nuevo DataFrame llamado 'df_atributos' que contiene los valores únicos de la columna 'attributes' del DataFrame original 'df_atribute', junto con una columna de identificación única para cada valor.**

In [ ]:
atributos = df_atribute["attributes"].unique()
df_atributos = pd.DataFrame(atributos, columns=["Descripcion"])
df_atributos['IdAtributos'] = df_atributos.index

**Exportamos tabla de atributos**

In [ ]:
df_atributos.to_csv("Datasets/Businessatributos.csv")

**Eliminamos las columnas "attributes" de aux**

In [ ]:
aux.drop(columns=["attributes"],inplace=True)

**Creamos la tabla de dimensiones "Hours" a partir de aux**

In [ ]:
df_hours = aux.loc[:, ["id_business",'hours']]

In [ ]:
df_hours

**Exportamos la tabla de Hours**

In [ ]:
df_hours.to_csv("Datasets/BusinessHoras.csv")

**Creamos el dataframe df_Hour_detalle**

In [ ]:
df_hour_detalle=df_b[['id_business','hours']]

**Desglosamos los diferentes horarios en diferentes filas para cada valor de forma individual**

In [ ]:
df_hour_detalle=df_hour_Detalle[['id_business','hours']].explode('hours')

**Creamos un dataframe llamado hour con las horas unicas de df_hour_detalle**

In [ ]:
hours=df_hour_detalle['hours'].unique()

**Se crea un nuevo DataFrame que contiene una sola columna llamada "Descripcion" que contiene los valores de la columna 'hours' y reindexamos**

In [ ]:
DescHour=pd.DataFrame(hours,columns=["Descripcion"])
DescHour['id_hour']=DescHour.index

**Exportamos el dataframe BusinessHorarios**

In [ ]:
DescHour.to_csv('businessHorarios.csv',index=False)

**Hacemos una union entre df_hour_detalle y DescHour**

In [ ]:
df_hour_detalle.merge(DescHour,left_on='hours',right_on='Descripcion',how='inner')[['id_business','id_hour']]

**Exportamos el dataframe df_hour_detalle**

In [ ]:
df_hour_detalle.to_csv('businessDetallesHora.csv',index=False)

**Creamos el dataframe df_detalleatributo a partir de la union entre df_atribute y df_atributos**

In [ ]:
df_detalleatributo = df_atribute.merge(df_atributos, left_on="attributes", right_on="Descripcion", how="inner")

**Eliminamos las columnas "attributes" y "Descripcion" de df_detalleatributo**

In [ ]:
df_detalleatributo.drop(columns = "attributes",inplace=True)
df_detalleatributo.drop(columns = "Descripcion",inplace=True)

**Exportamos dataframe df_detalleatributo**

In [ ]:
dfdetalleatributo.to_csv("Datasets/DetalleAtributos.csv")

**Eliminamos la columna hours de aux**

In [ ]:
aux.drop(columns=["hours"],inplace=True)

In [ ]:
aux.head(1)

## 1.4-tip.json

**Abrimos el archivo json**

In [219]:
json_objects=[]

with open('../tip.json', 'r',encoding='utf-8') as f:
    for line in f:
        json_objects.append(json.loads(line))


df_tip = pd.DataFrame(json_objects)

 **Obtenemos un DataFrame que contenga solo las filas correspondientes a los negocios presentes en BusinessYelp**

In [220]:
df_tip=df_tip[df_tip['business_id'].isin(BusinessYelp.business_id.unique().tolist())]

In [221]:
df_tip

,user_id,business_id,text,date,compliment_count


**Hacemos una unión df_tip con BusinessYelp**

In [ ]:
df_tip=BusinessYelp.merge(df_tip,left_on='business_id',right_on='business_id',how='right')[['user_id', 'id_business', 'text', 'date', 'compliment_count']]

**Hacemos una unión df_tip con UsuarioYelp**

In [ ]:
df_tip=dfusuario.merge(df_tip,left_on='yelp_id',right_on='user_id',how='right')[['id_user', 'id_business', 'text', 'date', 'compliment_count']]

**Exportamos dataframe tip.json**

In [ ]:
df_tip.to_csv('tipsYelp.csv',index=False)

## 1.5-review.json

**Leemos el archivo review.json de Yelp**

In [ ]:
jsonarr=[]
with open("../review.json", 'r',encoding='utf-8') as f:
    for line in f:
        jsonarr.append(json.loads(line))


df_review = pd.DataFrame(jsonarr)

**Eliminamos duplicados y reindexamos**

In [ ]:
df_review= df_review.drop_duplicates(subset=["review_id", "text"], keep="first")
df_review['id_review'] = df_review.index

**Creamos un dataframe con "id_review", "review_id" llamado df_ReviewYelp**

In [ ]:
df_ReviewYelp = df_review[["id_review", "review_id"]]

**Exportamos el dataframe ReviewYelpId**

In [ ]:
dfReviewYelp.to_csv("Datasets/ReviewYelpId.csv")

**Eliminamos columna review_id de df_Review**

In [ ]:
df_review.drop(columns = "review_id",inplace=True)

**Unimos el dataframe df_review con dfusuario**

In [ ]:
df_review = df_review.merge(dfusuario, left_on="user_id", right_on="yelp_id", how="inner")

**Creamos el dataframe df_reviewfinal a partir de df_review y BusinessYelp**

In [ ]:
df_reviewfinal = df_review.merge(BusinessYelp, left_on="business_id", right_on="businessYelp_id", how="inner")

**Se eliminan las columnas "user_id" y "yelp_id" del dataframe df_reviewfinal**

In [ ]:
df_reviewfinal.drop(columns = "business_id",inplace=True)
df_reviewfinal.drop(columns = "businessYelp_id",inplace=True)
df_reviewfinal.drop(columns = "Yelp_id",inplace=True)

**Exportamos dataframe review**

In [ ]:
df_reviewfinal.to_csv("Review.csv")

# 2-ETL de Archivos de Google Maps

## 2.1-Review Estados

**Leemos el archivo del Estado de California**

In [ ]:
#Funcion para leer el estado de California
def read_json_files(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                for line in file:
                    line = line.strip()
                    if line:
                        try:
                            json_obj = json.loads(line)
                            data.append(json_obj)
                        except json.JSONDecodeError:
                            continue

    df = pd.DataFrame(data)
    return df

In [ ]:
#Leemos cada json y lo convertimos en un dataframe y lo agregamos a una lista de dataframe
folder_path = "../reviews-estados/review-California"
df_list = []
df_california = read_json_files(folder_path)
df_list.append(df_california)

In [ ]:
#unimos todos los dataframes generados en uno
df_california_consolidado = pd.concat(df_list, ignore_index=True)

**Eliminamos columnas que no se usarán en el proyecto**

In [ ]:
#Dropeamos las columnas que no usaremos
df_california_consolidado.drop(["name", "pics", "resp"], axis=1, inplace=True)

**Cambiamos el formato de la columna time**

In [ ]:
# Convertir la columna "time" al formato adecuado
df_california_consolidado['time'] = pd.to_datetime(df_california_consolidado['time'], unit='ms')

**Creamos las columnas date y hour**

In [ ]:
#Creamos dos columnas (date y hour) para ser extraidos de la columna time y convertir al formato adecuado 
from datetime import datetime
df_california_consolidado['hour'] = pd.to_datetime(df_california_consolidado['time']).dt.strftime('%H:%M:%S')

df_california_consolidado['date'] = pd.to_datetime(df_california_consolidado['time']).dt.strftime('%Y-%m-%d')

**Se exporta el dataframe de California**

In [ ]:
# Exportar el DataFrame
df_california_consolidado.to_csv('california.csv', escapechar='\\', index=False)

**El mismo ETL se aplica a los estados de Florida, New York, Pensylvania y Texas**

## 2.2- Carpeta Metadata_Sitios

**Leemos la carpeta y la consolidamos en un archivo**

In [18]:
#Funcion para abrir carpeta de metadata-sitios
def merge_json_files(folder_path):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath) as file:
                for line in file:
                    try:
                        obj = json.loads(line)
                        merged_data.append(obj)
                    except json.JSONDecodeError as e:
                        print(f"Error al decodificar JSON en el archivo {filename}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
folder_path = '../metadata-sitios/'
Metadata= merge_json_files(folder_path)

**Abrimos el archivo del estado de california**

In [19]:
def merge_json_files(folder_path):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath) as file:
                for line in file:
                    try:
                        obj = json.loads(line)
                        merged_data.append(obj)
                    except json.JSONDecodeError as e:
                        print(f"Error al decodificar JSON en el archivo {filename}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
folder_path = '../reviews-estados/review-California'
df_california= merge_json_files(folder_path)

**Filtramos Metadata_Sitios con el estado de california**

In [20]:
# Filtrar los registros de Metadata que tienen el mismo 'gmap_id' que los de CaliforniaR
MetaCalifornia = Metadata[Metadata['gmap_id'].isin(df_california['gmap_id'])].copy()

# Agregar la nueva columna 'California' al nuevo DataFrame
MetaCalifornia['State'] = 'California'

**Eliminamos Filas con valores nulos**

In [21]:
# Eliminar filas con valores nulos en la columna 'category'
MetaCalifornia = MetaCalifornia.dropna(subset=['category'])

**Abrimos el archivo para el estado de Florida**

In [22]:
def merge_json_files(folder_path):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath) as file:
                for line in file:
                    try:
                        obj = json.loads(line)
                        merged_data.append(obj)
                    except json.JSONDecodeError as e:
                        print(f"Error al decodificar JSON en el archivo {filename}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
folder_path = '../reviews-estados/review-Florida/'
df_florida= merge_json_files(folder_path)

**Filtramos Metadata_Sitios con el estado de florida**

In [23]:
# Filtrar los registros de Metadata que tienen el mismo 'gmap_id' que los de CaliforniaR
MetaFlorida= Metadata[Metadata['gmap_id'].isin(df_florida['gmap_id'])].copy()

# Agregar la nueva columna 'California' al nuevo DataFrame
MetaFlorida['State'] = 'Florida'

**Eliminamos Filas con valores nulos**

In [24]:
# Eliminar filas con valores nulos en la columna 'category'
MetaFlorida = MetaFlorida.dropna(subset=['category'])

**Abrimos archivo de New york**

In [25]:
def merge_json_files(folder_path):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath) as file:
                for line in file:
                    try:
                        obj = json.loads(line)
                        merged_data.append(obj)
                    except json.JSONDecodeError as e:
                        print(f"Error al decodificar JSON en el archivo {filename}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
folder_path = '../reviews-estados/review-New_York'
df_new_york= merge_json_files(folder_path)

**Filtramos Metadata_Sitios con el estado de New york**

In [26]:
# Filtrar los registros de Metadata que tienen el mismo 'gmap_id' que los de CaliforniaR
MetaNewYork= Metadata[Metadata['gmap_id'].isin(df_new_york['gmap_id'])].copy()

# Agregar la nueva columna 'California' al nuevo DataFrame
MetaNewYork['State'] = 'New York'

**Eliminamos Filas con valores nulos**

In [27]:
# Eliminar filas con valores nulos en la columna 'category'
MetaNewYork = MetaNewYork.dropna(subset=['category'])

**Abrimos archivo de Texas**

In [28]:
def merge_json_files(folder_path):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath) as file:
                for line in file:
                    try:
                        obj = json.loads(line)
                        merged_data.append(obj)
                    except json.JSONDecodeError as e:
                        print(f"Error al decodificar JSON en el archivo {filename}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
folder_path = "../reviews-estados/review-Texas"
df_Texas= merge_json_files(folder_path)

**Filtramos Metadata_Sitios con el estado de New york**

In [29]:
# Filtrar los registros de Metadata que tienen el mismo 'gmap_id' que los de CaliforniaR
MetaTexas= Metadata[Metadata['gmap_id'].isin(df_Texas['gmap_id'])].copy()

# Agregar la nueva columna 'California' al nuevo DataFrame
MetaTexas['State'] = 'Texas'

**Eliminamos Filas con valores nulos**

In [30]:
# Eliminar filas con valores nulos en la columna 'category'
MetaTexas = MetaTexas.dropna(subset=['category'])

**Abrimos archivo de Pensylvania**

In [31]:
def merge_json_files(folder_path):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath) as file:
                for line in file:
                    try:
                        obj = json.loads(line)
                        merged_data.append(obj)
                    except json.JSONDecodeError as e:
                        print(f"Error al decodificar JSON en el archivo {filename}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
folder_path = "../reviews-estados/review-Pennsylvania"
df_Pensylvania= merge_json_files(folder_path)

**Filtramos Metadata_Sitios con el estado de Pensylvania**

In [32]:
# Filtrar los registros de Metadata que tienen el mismo 'gmap_id' que los de CaliforniaR
MetaPensylvania= Metadata[Metadata['gmap_id'].isin(df_Pensylvania['gmap_id'])].copy()

# Agregar la nueva columna 'California' al nuevo DataFrame
MetaPensylvania['State'] = 'Pennsylvania'

**Eliminamos Filas con valores nulos**

In [33]:
# Eliminar filas con valores nulos en la columna 'category'
MetaPensylvania = MetaPensylvania.dropna(subset=['category'])

**Concatenamos todos los estados**

In [34]:
# Lista de los DataFrames 
dataframes = [MetaCalifornia, MetaFlorida, MetaNewYork, MetaTexas, MetaPensylvania]

MetaEstados = pd.concat(dataframes)

**Eliminamos las columnas "Price" y "description"**

In [35]:
MetaEstados = MetaEstados.drop(['description', 'price'], axis=1)

**Exportamos el dataframe**

In [ ]:
# Exportar el DataFrame
MetaEstados.to_csv('MetaEstados.csv', escapechar='\\', index=False)